# 데이터 수집

## 2005년도 부터 2019년도 한국 제작 영화 데이터 수집
- 2005-01-01 ~ 2019-12-31 (코로나19 이전)
- KOBIS(=KOFIC OpenAPI)에서 '주간 박스오피스 Top10에 한 번이라도 등장'한 영화만 선별.
- 한국 제작만 필터.

In [ ]:
# -*- coding: utf-8 -*-
"""
2005-01-01 ~ 2019-12-31
KOBIS(=KOFIC OpenAPI)에서 '주간 박스오피스 Top10에 한 번이라도 등장'한 영화만 선별
→ 한국 제작만 필터 → TMDB 스타일 스키마 CSV 생성
(revenue = 국내 최종 누적매출, audience_total = 국내 최종 누적관객)

출력 열 순서:
movie_id,title,original_title,original_language,release_date,runtime,budget,
revenue,vote_average,vote_count,popularity,genres,production_companies,
production_countries,release_year,release_month,audience_total
"""

import os, time, json, requests
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from urllib.parse import urlencode
from tqdm import tqdm
from dotenv import load_dotenv

# ========= 기간/경로 설정 =========
START = date(2005, 1, 1)
END   = date(2019,12,31)

OUT_DIR       = "./data/raw"
WEEKLY_PICKLE = f"{OUT_DIR}/kobis_weekly_pool.pkl"      # 주간 스캔 결과 캐시
OUT_PARTIAL   = f"{OUT_DIR}/kobis_weekly_partial.csv"   # 메타 누적(재시작용)
OUT_FINAL     = f"{OUT_DIR}/kobis_weekly_raw.csv"     # 최종 TMDB형

# 하루에 처리할 movieInfo 개수(쿼터 따라 조정)
MOVIES_PER_RUN = 1000

# 요청/재시도
TIMEOUT = 20
RETRY   = 3
SLEEP   = 0.12

# ========= 인증 =========
load_dotenv()
KOBIS_KEY = os.getenv("KOFIC_API_KEY") or os.getenv("KOBIS_KEY")
BASE = "https://www.kobis.or.kr/kobisopenapi/webservice/rest"
os.makedirs(OUT_DIR, exist_ok=True)

# ========= 공통 HTTP =========
def kobis_get(path, params, retry=RETRY):
    q = {"key": KOBIS_KEY, **params}
    url = f"{BASE}{path}.json?{urlencode(q)}"
    last = None
    for i in range(retry):
        try:
            r = requests.get(url, timeout=TIMEOUT)
            if r.status_code in (429,500,502,503,504):
                time.sleep(min(2**i,8)); continue
            r.raise_for_status()
            data = r.json()
            if data.get("faultInfo"):
                fi = data["faultInfo"]
                raise RuntimeError(f"KOBIS fault: {fi.get('message')} (code={fi.get('errorCode')})\nURL: {url}")
            time.sleep(SLEEP)
            return data
        except Exception as e:
            last = e
            if i == retry-1: raise
            time.sleep(min(2**i,8))
    raise RuntimeError(last)

# ========= 유틸 =========
def sundays(start: date, end: date):
    # start 기준 그 주 일요일(weekGb=0 사용)
    s = start + timedelta(days=(6 - start.weekday()))
    while s <= end:
        yield s
        s += timedelta(days=7)

def to_iso_from_yyyymmdd(s):
    if not s: return pd.NA
    s = str(s)
    return f"{s[:4]}-{s[4:6]}-{s[6:]}" if len(s) >= 8 else pd.NA

def json_list_str(xs):
    if xs is None: return "[]"
    if isinstance(xs, list): return json.dumps(xs, ensure_ascii=False)
    return json.dumps([str(xs)], ensure_ascii=False)

# ========= 1) 주간 Top10 스캔: 등장 영화 + 최종 누적 =========
def weekly_scan():
    """
    주간 박스오피스(Top10) 전체 주차 스캔 → movieCd별 최댓값 누적매출/관객 및 기본 정보 모음
    반환: dict[movieCd] = {max_sales, max_audi, movieNm, openDt, weeks}
    """
    pool = {}  # movieCd -> dict
    for s in tqdm(list(sundays(START, END)), desc="Weekly scan (Top10)"):
        js = kobis_get("/boxoffice/searchWeeklyBoxOfficeList",
                       {"targetDt": s.strftime("%Y%m%d"), "weekGb": "0"})
        items = js.get("boxOfficeResult",{}).get("weeklyBoxOfficeList",[]) or []
        for it in items:
            cd  = it.get("movieCd")
            if not cd: continue
            nm  = it.get("movieNm")
            odt = it.get("openDt")  # yyyymmdd
            salesAcc = pd.to_numeric(it.get("salesAcc"), errors="coerce") or 0
            audiAcc  = pd.to_numeric(it.get("audiAcc"),  errors="coerce") or 0
            rec = pool.get(cd, {"max_sales":0, "max_audi":0, "movieNm":nm, "openDt":odt, "weeks":0})
            rec["max_sales"] = max(rec["max_sales"], salesAcc)
            rec["max_audi"]  = max(rec["max_audi"],  audiAcc)
            rec["weeks"]     = rec["weeks"] + 1
            if not rec.get("movieNm"):  rec["movieNm"] = nm
            if not rec.get("openDt"):   rec["openDt"]  = odt
            pool[cd] = rec
    pd.to_pickle(pool, WEEKLY_PICKLE)
    return pool

# ========= 2) movieInfo 메타(한국 제작 필터) + TMDB 스키마로 변환 =========
def fetch_movie_info(cd: str):
    js = kobis_get("/movie/searchMovieInfo", {"movieCd": cd})
    return js.get("movieInfoResult",{}).get("movieInfo",{}) or {}

def is_korean_production(info) -> bool:
    nations = [n.get("nationNm") for n in (info.get("nations") or []) if n.get("nationNm")]
    return any(str(n).strip() in {"한국","대한민국","Korea","South Korea","Republic of Korea"} for n in nations)

def to_tmdb_row(cd, pool_rec, info):
    movieNm   = info.get("movieNm") or pool_rec.get("movieNm")
    movieNmEn = info.get("movieNmEn") or movieNm
    openDt    = info.get("openDt") or pool_rec.get("openDt")
    iso       = to_iso_from_yyyymmdd(openDt)
    dt        = pd.to_datetime(iso, errors="coerce") if pd.notna(iso) else pd.NaT

    showTm    = pd.to_numeric(info.get("showTm"), errors="coerce")
    genres    = [g.get("genreNm")   for g in (info.get("genres")   or []) if g.get("genreNm")]
    companies = [c.get("companyNm") for c in (info.get("companys") or []) if c.get("companyNm")]

    return {
        "movie_id": cd,
        "title": movieNm,
        "original_title": movieNmEn,
        "original_language": "ko",
        "release_date": iso,
        "runtime": showTm,
        "budget": pd.NA,
        "revenue": float(pool_rec["max_sales"]),       # 국내 최종 누적매출(원)
        "vote_average": pd.NA,
        "vote_count": pd.NA,
        "popularity": pd.NA,
        "genres": json_list_str(genres),
        "production_companies": json_list_str(companies),
        "production_countries": json_list_str(["KR"]),
        "release_year": (dt.year  if pd.notna(dt) else pd.NA),
        "release_month": (dt.month if pd.notna(dt) else pd.NA),
        "audience_total": float(pool_rec["max_audi"]), # 국내 최종 누적관객
    }

# ========= 3) 실행(재시작 가능) =========
def main():
    if not KOBIS_KEY:
        raise SystemExit("❌ .env에 KOFIC_API_KEY(=KOBIS_KEY)가 없습니다.")

    # 1) 주간 스캔(캐시 활용)
    if os.path.exists(WEEKLY_PICKLE):
        pool = pd.read_pickle(WEEKLY_PICKLE)
    else:
        pool = weekly_scan()

    # 주간에 한 번이라도 잡힌 영화만(=누적매출 추적 가능)
    candidates = [cd for cd, rec in pool.items() if rec.get("max_sales",0) > 0]

    # 2) 이미 수집한 partial 읽기(재시작)
    done = set()
    if os.path.exists(OUT_PARTIAL):
        prev = pd.read_csv(OUT_PARTIAL, dtype=str)
        if "movie_id" in prev.columns:
            done = set(prev["movie_id"].astype(str).tolist())

    todo = [cd for cd in candidates if cd not in done]
    if not todo:
        print("✅ 모두 처리됨. 최종 파일만 정리합니다.")
    else:
        batch = todo[:MOVIES_PER_RUN]
        print(f"이번 배치: {len(batch)}편 / 남은 {len(todo)-len(batch)}편")

        rows = []
        for cd in tqdm(batch, desc="movieInfo + merge (KR only)"):
            try:
                info = fetch_movie_info(cd)
                # 한국 제작 필터 + 개봉일(기간 내 안전망)
                if not is_korean_production(info):
                    continue
                odt = info.get("openDt") or pool[cd].get("openDt")
                if not odt or len(str(odt)) != 8:
                    continue
                # 기간 안전망 (열린 날짜 기준)
                y, m, d = int(odt[:4]), int(odt[4:6]), int(odt[6:8])
                opened = date(y,m,d)
                if opened < START or opened > END:
                    continue

                row = to_tmdb_row(cd, pool[cd], info)
                rows.append(row)
            except Exception as e:
                print("skip", cd, e)

        if rows:
            df = pd.DataFrame(rows)
            header = not os.path.exists(OUT_PARTIAL)
            df.to_csv(OUT_PARTIAL, index=False, mode=("w" if header else "a"),
                      header=header, encoding="utf-8-sig")

    # 3) 최종 정리/저장(TMDB 스키마)
    cols = ["movie_id","title","original_title","original_language","release_date","runtime","budget",
            "revenue","vote_average","vote_count","popularity","genres","production_companies",
            "production_countries","release_year","release_month","audience_total"]

    part = pd.read_csv(OUT_PARTIAL, dtype=str) if os.path.exists(OUT_PARTIAL) else pd.DataFrame(columns=cols)
    if not part.empty:
        part["release_year"]   = pd.to_numeric(part["release_year"], errors="coerce").astype("Int64")
        part["release_month"]  = pd.to_numeric(part["release_month"], errors="coerce").astype("Int64")
        part["runtime"]        = pd.to_numeric(part["runtime"], errors="coerce")
        part["revenue"]        = pd.to_numeric(part["revenue"], errors="coerce")
        part["audience_total"] = pd.to_numeric(part["audience_total"], errors="coerce")
        part = part.drop_duplicates(subset=["movie_id"]).sort_values(["release_year","release_month","title"])

    part.reindex(columns=cols).to_csv(OUT_FINAL, index=False, encoding="utf-8-sig")
    print(f"✅ 저장: {OUT_FINAL} (rows={len(part):,})")

if __name__ == "__main__":
    main()


## 데이터 결측치 처리
- "budget", "vote_average", "vote_count", "popularity" 은 없으므로 예외

In [7]:
# kobis_weekly_clean.ipynb
import pandas as pd
import ast
from datetime import datetime
import os

PATH_IN  = "./data/raw/kobis_weekly_raw.csv"
PATH_OUT = "./data/raw/kobis_weekly_clean.csv"

df = pd.read_csv(PATH_IN, dtype={"movie_id": str})
print("원본 shape:", df.shape)

# 결측치 현황(전)
na_before = df.isna().sum().rename("na_before")

# 스키마 보강
for col in ["budget", "vote_average", "vote_count", "popularity"]:
    if col not in df.columns:
        df[col] = pd.NA

# 형 변환
num_cols = ["runtime", "revenue", "audience_total", "vote_average", "vote_count", "popularity"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

if "release_date" in df.columns:
    df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")

# 연/월 채움
if "release_year" not in df.columns:  df["release_year"]  = pd.NA
if "release_month" not in df.columns: df["release_month"] = pd.NA

mask_dt = df["release_date"].notna()
df.loc[df["release_year"].isna()  & mask_dt, "release_year"]  = df.loc[mask_dt, "release_date"].dt.year
df.loc[df["release_month"].isna() & mask_dt, "release_month"] = df.loc[mask_dt, "release_date"].dt.month

df["release_year"]  = pd.to_numeric(df["release_year"], errors="coerce").astype("Int64")
df["release_month"] = pd.to_numeric(df["release_month"], errors="coerce").astype("Int64")

# 장르 파싱 -> 리스트
def parse_genres(x):
    if isinstance(x, list): return [str(t).strip() for t in x if str(t).strip()]
    if pd.isna(x): return []
    s = str(x).strip()
    if s.startswith("[") and s.endswith("]"):
        try:
            val = ast.literal_eval(s)
            out = []
            for item in val:
                if isinstance(item, dict):
                    out.append(item.get("name") or item.get("genreNm") or str(item))
                else:
                    out.append(str(item).strip())
            return [g for g in out if g]
        except Exception:
            pass
    if "," in s:
        return [t.strip() for t in s.split(",") if t.strip()]
    return [s] if s else []

if "genres" not in df.columns:
    df["genres"] = "[]"
df["genres"] = df["genres"].apply(parse_genres)

# 결측치 현황(후) — 리스트 상태에서 계산
na_after = df.isna().sum().rename("na_after")
summary = pd.concat([na_before, na_after], axis=1).fillna(0).astype(int)
summary["delta"] = summary["na_after"] - summary["na_before"]

print("\n정리 후 shape:", df.shape)
print("\n컬럼별 결측치 변화:")
print(summary.sort_index())

# === 저장용: genres를 "['코미디', '드라마']" 형태의 문자열로 변환 ===
def list_to_literal_str(lst):
    # 내부 작은따옴표 이스케이프
    safe = [str(x).replace("'", "\\'") for x in (lst or [])]
    return "[" + ", ".join(f"'{x}'" for x in safe) + "]"

df_to_save = df.copy()
df_to_save["genres"] = df_to_save["genres"].apply(list_to_literal_str)

os.makedirs(os.path.dirname(PATH_OUT), exist_ok=True)
df_to_save.to_csv(PATH_OUT, index=False, encoding="utf-8-sig")
print(f"\n저장 완료 → {PATH_OUT}")


원본 shape: (927, 17)

정리 후 shape: (927, 17)

컬럼별 결측치 변화:
                      na_before  na_after  delta
audience_total                0         0      0
budget                      927       927      0
genres                        0         0      0
movie_id                      0         0      0
original_language             0         0      0
original_title                0         0      0
popularity                  927       927      0
production_companies          0         0      0
production_countries          0         0      0
release_date                  0         0      0
release_month                 0         0      0
release_year                  0         0      0
revenue                       0         0      0
runtime                       0         0      0
title                         0         0      0
vote_average                927       927      0
vote_count                  927       927      0

저장 완료 → ./data/raw/kobis_weekly_clean.csv


## 네이버 네티즌 평점, 비평가 평점 데이터 수집
- 네티즌 평점은 실관람객이 아닌 관람객도 평점을 남길 수 있음.
- 평론가의 평균 평점과 인원 수를 구함.
- 네이버에 직접 크롤링이 어려운 영화는 직접 검색을 통해 데이터 수집. `ratings.csv`

In [ ]:
# build_naver_ratings.ipynb

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import urllib.parse

# 🎬 네가 가진 영화 리스트
titles = []

base_url = "https://movie.naver.com"

results = []

for idx, title in enumerate(titles, 1):
    try:
        # 🔎 검색어 안전하게 인코딩
        query = urllib.parse.quote(title)
        search_url = f"https://movie.naver.com/movie/search/result.naver?query={query}&section=all&ie=utf8"

        # 검색 페이지 요청
        res = requests.get(search_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(res.text, "html.parser")

        # 검색 결과에서 첫 번째 영화 링크 추출
        movie_tag = soup.select_one(".search_list_1 ul li dt a")
        if not movie_tag:
            print(f"[{idx}/{len(titles)}] {title} → 검색 결과 없음")
            continue

        movie_url = base_url + movie_tag["href"]

        # 영화 상세 페이지 요청
        res = requests.get(movie_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(res.text, "html.parser")

        # 평점, 참여자 수 가져오기
        score_tag = soup.select_one(".star_score em")
        count_tag = soup.select_one(".score_total em")

        score = score_tag.text if score_tag else "N/A"
        count = count_tag.text.replace(",", "") if count_tag else "0"

        results.append({"제목": title, "평점": score, "참여자수": count})
        print(f"[{idx}/{len(titles)}] {title} → 평점 {score}, 참여자 {count}")

        # 🔒 차단 방지: 1~3초 랜덤 대기
        time.sleep(random.uniform(1, 3))

    except Exception as e:
        print(f"[ERROR] {title}: {e}")

# ✅ 데이터프레임 저장
df = pd.DataFrame(results)
df.to_csv("./data/raw/naver_ratings.csv", index=False, encoding="utf-8-sig")
print("완료! ./data/raw/naver_ratings.csv 저장됨.")


완료! naver_ratings.csv 저장됨.


## 영화 감독과 영화 배우 데이터 수집하기

In [ ]:
# build_kobis_with_credits.ipynb
# -*- coding: utf-8 -*-
"""
입력:  ./data/raw/kobis_weekly_clean.csv  (movie_id = KOBIS movieCd)
출력1: ./data/raw/kobis_credits_partial.csv   (진행 중 캐시)
출력2: ./data/raw/kobis_with_credits.csv   (원본 + 감독/배우 한글 이름 append)

- 감독: 1명(목록의 첫 감독)
- 배우: castOrd 기준 상위 10명 (없으면 리스트 순서 기준)
- 이름: 모두 한국어(peopleNm) 사용
- 재실행 시 이미 처리한 movie_id는 건너뛰고 이어서 수집
"""

import os, time, json, requests
import pandas as pd
from urllib.parse import urlencode
from tqdm import tqdm
from dotenv import load_dotenv

# ---------------- 설정 ----------------
IN_PATH   = "./data/raw/kobis_weekly_clean.csv"
OUT_DIR   = "./data/raw"
OUT_PART  = f"{OUT_DIR}/kobis_credits_partial.csv"     # movie_id, director_name_ko, cast_names_ko
OUT_FINAL = f"{OUT_DIR}/kobis_with_credits.csv"     # 원본 + 붙이기 결과

CAST_TOP_N   = 10
MOVIES_PER_RUN = 1000         # 하루 쿼터 고려, 한 번에 처리할 최대 편수
SLEEP   = 0.12               # 호출 간 딜레이
TIMEOUT = 20
RETRY   = 3

# -------------- 인증/엔드포인트 --------------
load_dotenv()
KOBIS_KEY = os.getenv("KOFIC_API_KEY") or os.getenv("KOBIS_KEY")
BASE = "https://www.kobis.or.kr/kobisopenapi/webservice/rest"

os.makedirs(OUT_DIR, exist_ok=True)

# -------------- 공통 HTTP --------------
def kobis_get(path, params, retry=RETRY):
    if not KOBIS_KEY:
        raise SystemExit("❌ .env에 KOFIC_API_KEY(=KOBIS_KEY)가 없습니다.")
    q = {"key": KOBIS_KEY, **params}
    url = f"{BASE}{path}.json?{urlencode(q)}"
    last = None
    for i in range(retry):
        try:
            r = requests.get(url, timeout=TIMEOUT)
            if r.status_code in (429,500,502,503,504):
                time.sleep(min(2**i, 8)); continue
            r.raise_for_status()
            data = r.json()
            if data.get("faultInfo"):
                fi = data["faultInfo"]
                # 메시지와 URL을 함께 보여주되, 진행은 중단/스킵
                raise RuntimeError(f"KOBIS fault: {fi.get('message')} (code={fi.get('errorCode')})\nURL: {url}")
            time.sleep(SLEEP)
            return data
        except Exception as e:
            last = e
            if i == retry - 1:
                raise
            time.sleep(min(2**i, 8))
    raise RuntimeError(last)

# -------------- 단일 영화 크레딧 수집 --------------
def safe_int(x, default=999999):
    try:
        return int(x)
    except Exception:
        return default

def fetch_credits_kor(movie_cd: str):
    """KOBIS movieCd로 감독 1명 + 배우 상위 10명(한글명) 추출"""
    js = kobis_get("/movie/searchMovieInfo", {"movieCd": movie_cd})
    info = js.get("movieInfoResult", {}).get("movieInfo", {}) or {}

    # 감독(목록 첫 감독 1명)
    directors = info.get("directors") or []
    director_ko = ""
    if directors:
        # peopleNm: 한글명, peopleNmEn: 영문명
        director_ko = (directors[0].get("peopleNm") or "").strip()

    # 배우(castOrd 오름차순 상위 N)
    actors = info.get("actors") or []
    # castOrd가 없거나 비면 큰 수로 처리 → 리스트 순서 보존
    actors_sorted = sorted(
        actors,
        key=lambda a: safe_int(a.get("castOrd"), default=999999)
    )
    cast_top = []
    for a in actors_sorted[:CAST_TOP_N]:
        nm = (a.get("peopleNm") or "").strip()  # 한글명
        if nm:
            cast_top.append(nm)
    cast_names_ko = "|".join(cast_top)

    return {
        "movie_id": movie_cd,
        "director_name_ko": director_ko,
        "cast_names_ko": cast_names_ko
    }

# -------------- 메인 --------------
def main():
    # 입력 로드
    base = pd.read_csv(IN_PATH, dtype={"movie_id": str})
    if "movie_id" not in base.columns:
        raise SystemExit("❌ 입력 CSV에 movie_id 컬럼이 없습니다.")

    # partial(진행 캐시) 불러오기
    if os.path.exists(OUT_PART):
        part = pd.read_csv(OUT_PART, dtype={"movie_id": str})
    else:
        part = pd.DataFrame(columns=["movie_id", "director_name_ko", "cast_names_ko"])

    done_ids = set(part["movie_id"].tolist())
    all_ids  = [mid for mid in base["movie_id"].astype(str).tolist() if isinstance(mid, str)]
    todo_ids = [mid for mid in all_ids if mid not in done_ids]

    if not todo_ids:
        print("✅ 새로 수집할 대상 없음. 병합만 수행합니다.")
    else:
        batch = todo_ids[:MOVIES_PER_RUN]
        print(f"이번 배치: {len(batch)}편 / 남은 {len(todo_ids) - len(batch)}편")

        rows = []
        for mid in tqdm(batch, desc="KOBIS credits (ko)"):
            try:
                rows.append(fetch_credits_kor(mid))
            except Exception as e:
                # 에러는 스킵하고 진행(쿼터 초과/간헐 오류 등)
                print("skip", mid, e)

        if rows:
            add = pd.DataFrame(rows)
            # partial에 append (중복 제거)
            merged = pd.concat([part, add], ignore_index=True)
            merged = merged.drop_duplicates(subset=["movie_id"], keep="last")
            merged.to_csv(OUT_PART, index=False, encoding="utf-8-sig")
            part = merged

    # 최종 병합(원본 + credits)
    final = base.merge(part, on="movie_id", how="left")
    final.to_csv(OUT_FINAL, index=False, encoding="utf-8-sig")
    print(f"✅ 저장 완료: {OUT_FINAL} (rows={len(final):,})")
    # 진행 상황 참고(옵션)
    got = final["director_name_ko"].notna().sum()
    print(f"   └ 감독/배우 채워진 행: {got:,} / {len(final):,}")

if __name__ == "__main__":
    main()


이번 배치: 927편 / 남은 0편


KOBIS credits (ko):  41%|████▏     | 384/927 [05:16<07:34,  1.19it/s]

## 기존 파일에 네이버 네티즌 평점 / 비평가 평점 컬럼 붙이기

In [5]:
# build_kobis_with_ratings.ipynb
# -*- coding: utf-8 -*-

import os, pandas as pd

kobis_path  = "./data/raw/kobis_with_credits.csv"      # 방금 올린 파일
naver_path  = "./data/raw/naver_ratings.csv"           # 네이버 스크랩 결과
out_path    = "./data/raw/kobis_with_ratings.csv"

kobis = pd.read_csv(kobis_path)
naver = pd.read_csv(naver_path)

# 제목 정규화(공백 제거 등)로 매칭률 올리기
def norm(s): return str(s).strip().replace(" ", "")
kobis["__t"] = kobis["title"].map(norm)
naver["__t"] = naver["제목"].map(norm)

# 네이버 컬럼 영문으로 리네임
naver_ren = naver.rename(columns={
    "네티즌점수": "vote_average_naver",
    "네티즌평가수": "vote_count_naver",
    "전문가평점평균": "critic_average",
    "전문가수": "critic_count",
})

# 필요한 컬럼만 가져와 병합
merged = kobis.merge(
    naver_ren[["__t","vote_average_naver","vote_count_naver","critic_average","critic_count"]],
    on="__t", how="left"
)

# 기존 TMDB 스타일 컬럼 채우기(없으면 생성)
if "vote_average" in merged.columns:
    merged["vote_average"] = merged["vote_average"].fillna(merged["vote_average_naver"])
else:
    merged["vote_average"] = merged["vote_average_naver"]

if "vote_count" in merged.columns:
    merged["vote_count"] = merged["vote_count"].fillna(merged["vote_count_naver"])
else:
    merged["vote_count"] = merged["vote_count_naver"]

# 작업용 키 제거 후 저장
merged = merged.drop(columns=["__t"])
os.makedirs(os.path.dirname(out_path), exist_ok=True)
merged.to_csv(out_path, index=False, encoding="utf-8-sig")

print("saved:", out_path,
      "| rows:", len(merged),
      "| matched:", merged["vote_average_naver"].notna().sum())


saved: ./data/raw/kobis_with_ratings.csv | rows: 927 | matched: 925


## 장르 Explode
- 영화 장르별 분석을 위해 explode

In [7]:
import os, ast, pandas as pd

# 1) 원본 데이터 불러오기
in_path = "./data/processed/kobis_weekly_final.csv"
if not in_path:
    raise FileNotFoundError("입력 CSV를 찾을 수 없어요. 실제 저장 경로(언더스코어 여부 포함)를 확인하세요.")

print("사용 경로:", in_path)
df_raw = pd.read_csv(in_path, dtype={"movie_id": str})

# 2) genres를 리스트로 파싱
def parse_genres(x):
    if isinstance(x, list): return x
    if pd.isna(x): return []
    s = str(x).strip()
    if s.startswith("[") and s.endswith("]"):
        try:
            vals = ast.literal_eval(s)
            out = []
            for v in vals:
                if isinstance(v, dict):
                    out.append(v.get("name") or v.get("genreNm") or str(v))
                else:
                    out.append(str(v).strip())
            return [g for g in out if g]
        except Exception:
            pass
    if "," in s:
        return [t.strip() for t in s.split(",") if t.strip()]
    return [s] if s else []

df_raw["genres"] = df_raw["genres"].apply(parse_genres)

# 3) 필요 시 explode (옵션)
df_exploded = df_raw.explode("genres").query("genres.notna() and genres.str.strip() != ''")
print("원본/폭발 후:", df_raw.shape, "→", df_exploded.shape)

# 4) 저장 예시
df_exploded.to_csv("./data/processed/kobis_genre_exploded.csv", index=False, encoding="utf-8-sig")


사용 경로: ./data/processed/kobis_weekly_final.csv
원본/폭발 후: (927, 23) → (1554, 23)
